In [1]:
%cd ..
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from matplotlib import pyplot as plt
from rx import operators

from configs.config import GlobalConfig
GlobalConfig.initialize_global_configuration()

C:\Users\wuyua\Projects\PycharmProjects\RxExperiments


In [2]:
import numpy as np
import logging
from utils.logging import configure_logger_to_output
logging_output = configure_logger_to_output(level=logging.INFO)
logging_output

Output()

In [3]:
from controls.mqtt_fp50 import MQTTFP50Control, FP50Command
from utils.mqtt_wrapper import MQTTClientWrapper

client = MQTTClientWrapper(client_id="fp50_test", clean_session=False)
client.connect("192.168.43.1")
client.loop_start()

In [4]:
control = MQTTFP50Control(client)

In [5]:
control.subscribe(lambda x: logging.info(x))

In [6]:
control.on_command(FP50Command(31))

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\wuyua\anaconda3\envs\RxExperiments\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\wuyua\anaconda3\envs\RxExperiments\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\wuyua\anaconda3\envs\RxExperiments\lib\site-packages\paho\mqtt\client.py", line 3452, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "C:\Users\wuyua\anaconda3\envs\RxExperiments\lib\site-packages\paho\mqtt\client.py", line 1779, in loop_forever
    rc = self.loop(timeout, max_packets)
  File "C:\Users\wuyua\anaconda3\envs\RxExperiments\lib\site-packages\paho\mqtt\client.py", line 1181, in loop
    rc = self.loop_read(max_packets)
  File "C:\Users\wuyua\anaconda3\envs\RxExperiments\lib\site-packages\paho\mqtt\client.py", line 1572, in loop_read
    rc = self._packet_read()
  File "C:\Users\wuyua\anaconda3\envs\RxExperiments\lib\site-p

In [6]:
from sink.visualization_sink import PlotlyVisualizationSink
from IPython.display import display
from plotly import graph_objects as go
import pandas as pd
sink = PlotlyVisualizationSink()

control.pipe(
    operators.sample(2),
    operators.map(lambda x: pd.DataFrame.from_dict([x.dict()])),
    operators.scan(lambda acc, new: acc.append(new), pd.DataFrame()),
    operators.map(lambda x: [go.Scatter(y=x.temperature), go.Scatter(y=x.power), go.Scatter(y=x.setpoint)])
).subscribe(sink)



display(sink.figure)

FigureWidget({
    'data': [], 'layout': {'template': '...', 'title': {'text': 'plot'}}
})

In [6]:
from operators.pid import pid, PIDInput
from rx import disposable
from source.ds18_source import DS18Source
ds18 = DS18Source(client).pipe(operators.share())
d = disposable.SerialDisposable()


In [7]:
P = 0.7
I = 0.0025
D = -43

In [8]:
from sink.visualization_sink import PlotlyVisualizationSink
from sink.save_data_sink import SaveDataSink
from IPython.display import display
from plotly import graph_objects as go
import pandas as pd
from operators import data_framer
import datetime
import rx
sink = PlotlyVisualizationSink()
save_sink = SaveDataSink("fp50_sine_20min_freq", "data");
src = rx.combine_latest(control, ds18).pipe(
    operators.sample(2),
    operators.map(lambda x: pd.DataFrame(index=[datetime.datetime.now()], data=[{"ds18": x[1], **x[0].dict()}])),
)

src.pipe(
    data_framer.data_framer(),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.temperature), go.Scatter(x=x.index, y=x.ds18), go.Scatter(x=x.index, y=x.setpoint)])
).subscribe(sink)

src.subscribe(save_sink)



display(sink.figure)

FigureWidget({
    'data': [], 'layout': {'template': '...', 'title': {'text': 'plot'}}
})

In [9]:
rx.just(True).pipe(
    operators.map(lambda x: control.on_command(FP50Command(60.0))),
    operators.delay(800),
    operators.map(lambda x: control.on_command(FP50Command(50.0))),
    operators.delay(800),
    operators.map(lambda x: control.on_command(FP50Command(40.0))),
    operators.delay(800),
    operators.map(lambda x: control.on_command(FP50Command(50.0))),
    operators.delay(800),
    operators.map(lambda x: control.on_command(FP50Command(60.0))),
).subscribe()


In [9]:
#sine
d.set_disposable(rx.interval(5).pipe(
    operators.map(lambda x: 10*np.sin(5*x*2*np.pi*1/(20*60))+50),
    operators.map(lambda x: FP50Command(x))
).subscribe(control))

In [9]:
def cl(setpoint):
    src = rx.combine_latest(control, ds18).pipe(
        operators.sample(2)
    )
    src.pipe
    d.set_disposable(src.pipe(
        operators.map(lambda x: PIDInput(setpoint, x[1])),
        pid(P, I, D, 2, limit=20, linearization_point=setpoint),
        operators.map(lambda x: FP50Command(x)),
    ).subscribe(control))
    
rx.just(True).pipe(
    operators.map(lambda x: cl(60)),
    operators.delay(800),
    operators.map(lambda x: cl(50)),
    operators.delay(800),
    operators.map(lambda x: cl(40)),
    operators.delay(800),
    operators.map(lambda x: cl(50)),
    operators.delay(800),
    operators.map(lambda x: cl(60)),
).subscribe()


In [1]:
setpoint = 60
import rx
src = rx.combine_latest(control, ds18).pipe(
    operators.sample(2)
)
src.pipe
d.set_disposable(src.pipe(
    operators.map(lambda x: PIDInput(setpoint, x[1])),
    pid(P, I, D, 2, limit=4, linearization_point=setpoint),
    operators.map(lambda x: FP50Command(x)),
).subscribe(control))

NameError: name 'control' is not defined

In [13]:
setpoint = 50
import rx

d.set_disposable(rx.combine_latest(control, ds18).pipe(
    operators.sample(2),
    operators.map(lambda x: PIDInput(setpoint, x[1])),
    pid(P, I, D, 2, limit=4, linearization_point=setpoint),
    operators.map(lambda x: FP50Command(x)),
).subscribe(control))